# Setup

## Python Librairies 📚

In [1]:
%pip install --quiet \
    langchain==0.0.309 \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client \
    apache-beam \
    datasets \
    tiktoken \
    "ipywidgets>=7,<8" \
    matplotlib \
    boto3 \
    chromadb==0.4.15

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.31.9 requires botocore==1.33.9, but you have botocore 1.34.34 which is incompatible.
awscli 1.31.9 requires s3transfer<0.9.0,>=0.8.0, but you have s3transfer 0.10.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.12.1 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.12.1 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.4 which is incompatible.
panel 0.13.1 requires bokeh<2.

## IAM Permissions 🔐

We recommend thhat the IAM Execution role for the notebook to be allowed to invoke the models. For instance, the following policy might be added. For users whishing to use Titan Embeddings `amazon.titan-embed-text-v1`

```json
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Sid": "VisualEditor0",
			"Effect": "Allow",
			"Action": "bedrock:InvokeModel",
			"Resource": "arn:aws:bedrock:*::foundation-model/amazon.titan-embed-text-v1"
		}
	]
}
```


A less restrictive policy is the `AmazonBedrockFullAccess` policy, managed by AWS.

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockAll",
            "Effect": "Allow",
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*"
        },
        {
            "Sid": "DescribeKey",
            "Effect": "Allow",
            "Action": [
                "kms:DescribeKey"
            ],
            "Resource": "arn:*:kms:*:::*"
        },
        {
            "Sid": "APIsWithAllResourceAccess",
            "Effect": "Allow",
            "Action": [
                "iam:ListRoles",
                "ec2:DescribeVpcs",
                "ec2:DescribeSubnets",
                "ec2:DescribeSecurityGroups"
            ],
            "Resource": "*"
        },
        {
            "Sid": "PassRoleToBedrock",
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "arn:aws:iam::*:role/*AmazonBedrock*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": [
                        "bedrock.amazonaws.com"
                    ]
                }
            }
        }
    ]
}
```

## Foundation model access (EULA) 📝

Accessing Foundation models is subject to agreeing to model providers EULA __beforehand__ in the console.

<img src="imgs/BedrockModelAccess.gif" width="750" align="center">


## Boto3

Boto3 contains numerous Python clients in order to interact with Amazon Bedrock, including:
* bedrock: to list foundation models and custom models
* bedrock-runtime: to invoke _FMs_
* bedrock-agent-runtime : to interact with a Knowledge Base (search and retrieve)

Ensure to have a recent version of _boto3_ beforehand.

In [6]:
import boto3
bedrock = boto3.client('bedrock')

Let's try with a simple listing example of foundation models, containing `TEXT` modality and provided by `Anthropic` model provider

In [10]:
import pandas as pd
res = bedrock.list_foundation_models(byOutputModality='TEXT', byProvider='Anthropic')
pd.DataFrame(res['modelSummaries'])

,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
0,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-instant-v1:2:100k,Claude Instant,Anthropic,[TEXT],[TEXT],True,[],[PROVISIONED],{'status': 'ACTIVE'}
1,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-instant-v1,Claude Instant,Anthropic,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
2,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-v1,Claude,Anthropic,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'LEGACY'}
3,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-v2:0:18k,Claude,Anthropic,[TEXT],[TEXT],True,[],[PROVISIONED],{'status': 'ACTIVE'}
4,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-v2:0:100k,Claude,Anthropic,[TEXT],[TEXT],True,[],[PROVISIONED],{'status': 'ACTIVE'}
5,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-v2:1:18k,Claude,Anthropic,[TEXT],[TEXT],True,[],[PROVISIONED],{'status': 'ACTIVE'}
6,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-v2:1:200k,Claude,Anthropic,[TEXT],[TEXT],True,[],[PROVISIONED],{'status': 'ACTIVE'}
7,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-v2:1,Claude,Anthropic,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
8,arn:aws:bedrock:us-east-1::foundation-model/an...,anthropic.claude-v2,Claude,Anthropic,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}


💡 In order to ensure targeting the same region, we'd advocate for setting up a Session, and derive a client. For instance:

In [4]:
import boto3
runtime_in_us_east1 = boto3.Session(region_name="us-east-1").client('bedrock-runtime')